In [1]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("/home/rafael/git/projeto_bom_pagador/dataset/train.csv", index_col='ID')

In [3]:
df.rename(columns={'LIMIT_BAL': 'Renda Anual', 'SEX':'Sexo','EDUCATION':'Educacao','MARRIAGE':'Estado Civil', 'AGE':'Idade','PAY_0': 'Tempo Emp.',"default payment next month":"Situacao"}, inplace=True)

In [4]:
df = df[['Renda Anual', 'Sexo', 'Educacao', 'Estado Civil', 'Idade','Tempo Emp.', 'Situacao']]

In [5]:
idx = df[df['Educacao'] > 4].index
df['Educacao'].loc[idx] = 4

In [6]:
idx =df[(df['Estado Civil'] < 1) | (df['Estado Civil'] > 3)].index
df['Estado Civil'].loc[idx] = 3

In [7]:
idx = df[df['Tempo Emp.']<0].index
df['Tempo Emp.'].loc[idx] = 0

In [8]:
df

,Renda Anual,Sexo,Educacao,Estado Civil,Idade,Tempo Emp.,Situacao
ID,,,,,,,
1,20000,2,2,1,24,2,1
2,120000,2,2,2,26,0,1
3,90000,2,2,2,34,0,0
4,50000,2,2,1,37,0,0
5,50000,1,2,1,57,0,0
...,...,...,...,...,...,...,...
20996,50000,1,2,1,49,1,1
20997,230000,1,2,2,38,2,0
20998,80000,1,2,1,38,0,0


In [9]:
for x in df.index:
    df['Tempo Emp.'].loc[x] = random.randrange(1,10)

In [10]:
df['Situacao'].value_counts()

0    16204
1     4796
Name: Situacao, dtype: int64

In [11]:
# Balanceando a Base

ap_idx = df[df['Situacao']==0].iloc[0:7194].index
rep_idx = df[df['Situacao']==1].index

In [12]:
df2 = pd.concat([df.loc[ap_idx], df.loc[rep_idx]])
df2

,Renda Anual,Sexo,Educacao,Estado Civil,Idade,Tempo Emp.,Situacao
ID,,,,,,,
3,90000,2,2,2,34,1,0
4,50000,2,2,1,37,9,0
5,50000,1,2,1,57,1,0
6,50000,1,1,2,37,8,0
7,500000,1,1,2,29,4,0
...,...,...,...,...,...,...,...
20987,30000,1,2,1,45,5,1
20992,210000,1,3,1,38,6,1
20993,460000,1,1,2,44,3,1


Criando datasets para consultas internas e externas.

In [13]:
df2.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_original.csv")

consul_int, consul_ext = train_test_split(df2, test_size=0.5)

consul_int.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_interno.csv")
consul_ext.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_externo.csv")

Trabalhando com o modelo de classificação

In [58]:
x = df2.drop(['Situacao'], axis=1)
y = df2['Situacao']

In [59]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [60]:
clf = RandomForestClassifier(max_depth=15, random_state=0)
clf.fit(x_train, y_train)
resul = clf.predict(x_test)
accuracy_score(y_test, resul)*100

60.05004170141785

In [39]:
x.columns

Index(['Renda Anual', 'Sexo', 'Educacao', 'Estado Civil', 'Idade',
       'Tempo Emp.'],
      dtype='object')

In [53]:
teste = []

In [54]:
for t in x.index:
    arr = list(x.loc[t])
    result = clf.predict([arr])
    sit = "Aprovado" if result[0] == 0 else "Reprovado"
    print("O cliente foi ", sit)
    teste.append(result)

O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente fo

O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente

O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente

O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente fo

O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi 

...

O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente fo

O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente

O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente f

O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente

O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Reprovado
O cliente foi  Aprovado
O cliente f

In [57]:
accuracy_score(y, teste)*100

82.32693911592995